## Setup - Llama.cpp

If you want to use [llama.cpp](https://github.com/abetlen/llama-cpp-python) on a MacBook M1 or M2, run the code below. For more options, check out the [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) docs.

!CMAKE_ARGS="-DLLAMA_METAL=on" pip install llama-cpp-python

# Linux and Mac
!CMAKE_ARGS="-DGGML_BLAS=ON -DGGML_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

In [2]:
# Download the model weight from huggingface
!huggingface-cli download motherduckdb/DuckDB-NSQL-7B-v0.1-GGUF DuckDB-NSQL-7B-v0.1-q8_0.gguf --local-dir . --local-dir-use-symlinks False


/home/rotundchonk/miniconda3/envs/duck-nql/lib/python3.12/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
DuckDB-NSQL-7B-v0.1-q8_0.gguf: 100%|███████| 7.16G/7.16G [04:15<00:00, 28.0MB/s]
Download complete. Moving file to DuckDB-NSQL-7B-v0.1-q8_0.gguf
DuckDB-NSQL-7B-v0.1-q8_0.gguf


In [3]:
!pip install wurlitzer pandas duckdb==0.9.2

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached duckdb-0.9.2.tar.gz (10.7 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 15.0 MB/s eta 0:00:0000:010:01
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [3937 lines of output]
      /home/rotundchonk/miniconda3/envs/duck-nql/lib/python3.12/site-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_buil

In [5]:
!conda install conda-forge::python-duckdb

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/rotundchonk/miniconda3/envs/duck-nql

  added / updated specs:
    - conda-forge::python-duckdb


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python-duckdb-1.2.0        |  py312h2ec8cdc_0        22.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        22.7 MB

The following NEW packages will be INSTALLED:

  python-duckdb      conda-forge/linux-64::python-duckdb-1.2.0-py312h2ec8cdc_0 


Proceed ([y]/n)? 
CondaSystemExit: 
Operation aborted.  Exiting.

^C


## Setup - General imports

In [1]:
import duckdb
from wurlitzer import pipes
from utils import generate_sql
from llama_cpp import Llama

## Load Model 

In [2]:
with pipes() as (out, err):
    client = Llama(
        model_path="DuckDB-NSQL-7B-v0.1-q8_0.gguf",
        n_ctx=2048,
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


## Connect to DuckDB

In [3]:
con = duckdb.connect("nyc.duckdb")

## Ask Question

In [4]:
question = "get all columns from taxi table starting with 'a'"

In [5]:
%%time
sql_query = generate_sql(question, con, client)

 SELECT COLUMNS('a.*') FROM taxi;
CPU times: user 12min 9s, sys: 51.4 s, total: 13min 1s
Wall time: 5min 25s


## Run Query on DuckDB

In [6]:
con.execute(sql_query).fetchdf()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2022-11-04 00:51:52,2022-11-04 01:02:08,1.0,2.37,1.0,N,237,186,1,10.0,0.5,0.5,2.00,0.0,0.3,15.80,2.5,0.00
1,2022-11-04 00:25:29,2022-11-04 00:39:51,5.0,2.95,1.0,N,48,79,1,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5,0.00
2,2022-11-04 00:43:21,2022-11-04 00:54:51,5.0,3.23,1.0,N,79,140,1,11.5,0.5,0.5,3.06,0.0,0.3,18.36,2.5,0.00
3,2022-11-04 00:05:49,2022-11-04 00:21:23,1.0,2.40,1.0,N,234,45,1,12.0,0.5,0.5,3.16,0.0,0.3,18.96,2.5,0.00
4,2022-11-04 00:35:49,2022-11-04 00:35:53,1.0,0.02,1.0,N,132,132,4,-2.5,-0.5,-0.5,0.00,0.0,-0.3,-5.05,0.0,-1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-11-04 00:40:37,2022-11-04 00:48:56,1.0,1.46,1.0,N,170,48,1,7.5,0.5,0.5,2.00,0.0,0.3,13.30,2.5,0.00
996,2022-11-04 00:57:24,2022-11-04 01:27:29,1.0,7.46,1.0,N,48,217,1,27.0,0.5,0.5,2.00,0.0,0.3,32.80,2.5,0.00
997,2022-11-04 01:29:40,2022-11-04 01:56:05,1.0,5.97,1.0,N,230,223,1,21.5,0.5,0.5,0.40,0.0,0.3,25.70,2.5,0.00
998,2022-11-04 01:44:59,2022-11-04 01:53:23,1.0,2.21,1.0,N,163,137,2,9.0,0.5,0.5,0.00,0.0,0.3,12.80,2.5,0.00


In [2]:
!pip install ollama

In [8]:
!ollama pull  duckdb-nsql:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 08a9f55588ea... 100% ▕████████████████▏ 3.8 GB                         
pulling cc23c3fcb197... 100% ▕████████████████▏  230 B                         
pulling bbede59a7c1c... 100% ▕████████████████▏  335 B                         
verifying sha256 digest 
writing manifest 
success 


In [10]:
import ollama

r = ollama.generate(
    #model='duckdb-nsql:7b',
    model='duckdb-nsql',
    system='''Here is the database schema that the SQL query will run on:
CREATE TABLE taxi (
    VendorID bigint,
    tpep_pickup_datetime timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count double,
    trip_distance double,
    fare_amount double,
    extra double,
    tip_amount double,
    tolls_amount double,
    improvement_surcharge double,
    total_amount double,
);''',
    prompt='get all columns ending with _amount from taxi table',
)

print(r['response'])

ResponseError: model requires more system memory (8.4 GiB) than is available (8.0 GiB) (status code: 500)